In [ ]:
te = @__DIR__

pk = dirname(te)

sr = joinpath(pk, "src")

;

In [ ]:
symdiff(
    (splitext(na)[1] for na in readdir(sr) if endswith(na, r"\.jl$") && na != "BioLab.jl"),
    (splitext(na)[1] for na in readdir(te) if endswith(na, r"\.ipynb$") && na != "runtests.ipynb"),
)

In [ ]:
using BioLab

In [ ]:
BioLab.TE

In [ ]:
BioLab.CA_

In [ ]:
BioLab.RA

In [ ]:
BioLab.IPYNB.run(@__DIR__, ("$pr.ipynb" for pr in ("runtests",)))